In [15]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.applications import VGG16
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.applications import VGG19
from keras.preprocessing import image
from keras.models import Model
#from keras import applications


In [11]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (3,img_width, img_height)
# Размер мини-выборки
#batch_size = 64
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750



In [6]:
resnet = ResNet50(weights='imagenet', 
                  include_top=False,input_shape=(200,200,3))

In [3]:
vgg16_net = VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape=(150, 150, 3))

In [3]:
vgg19_net = VGG19(weights='imagenet', 
                  include_top=False, 
                  input_shape=(150, 150, 3))

In [4]:
model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [51]:
model = Sequential()
model.add(vgg19_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [13]:
# Загружаем сеть VGG16 без части, которая отвечает за классификацию
base_model = VGG16(weights='imagenet', include_top=False)

In [16]:
# Добавляем слои классификации
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# Выходной слой с двумя нейронами для классов "кот" и "собака"
predictions = Dense(2, activation='softmax')(x)

In [17]:

# Составляем сеть из двух частей
model = Model(inputs=base_model.input, outputs=predictions)

In [18]:
# "Замораживаем" сверточные уровни сети VGG16
# Обучаем только вновь добавленные слои
for layer in base_model.layers:
    layer.trainable = False

In [5]:
#from tensorflow.python.keras.applications import ResNet50
#from tensorflow.python.keras.models import Sequential
#from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
#import tensorflow as tf
#tf.reset_default_graph() 
num_classes = 2
#resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
#my_new_model.add(ResNet50(include_top=False, pooling='avg'))
my_new_model.add(Dense(num_classes, activation='softmax',input_shape=(200,200,3)))

# Say not to train first layer (ResNet) model. It is already trained
#my_new_model.layers[0].trainable = False

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total para

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3), 
              metrics=['accuracy'])

In [20]:
# Каталог с данными для обучения
train_dir = r'C:\Users\Dmitry\Desktop\CNN_Research_2018\cats_vs_dogs\train'
# Каталог с данными для проверки
val_dir = r'C:\Users\Dmitry\Desktop\CNN_Research_2018\cats_vs_dogs\validation'
# Каталог с данными для тестирования
test_dir = r'C:\Users\Dmitry\Desktop\CNN_Research_2018\cats_vs_dogs\test'

In [21]:
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 17500 images belonging to 2 classes.
Found 3750 images belonging to 2 classes.
Found 3750 images belonging to 2 classes.


In [22]:
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [24]:
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size,callbacks=callbacks_list)

Epoch 1/2


ValueError: Error when checking target: expected dense_4 to have shape (2,) but got array with shape (1,)

In [11]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Доля правильных ответов на тестовых данных: %.2f%%" % (scores[1]*100))

Доля правильных ответов на тестовых данных: 51.02%


In [14]:
model.save('my_model.h5')

In [47]:
del model


In [4]:
from keras.models import load_model
model = load_model('my_model.h5')